### Weather Disaster Prediction 
#### DS 3000 Final Project
Members: Luke Abbatessa, Daniel Gilligan, Ruby Potash, Megan Putnam 

**Data Information**

Data set: https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/

Datasets include three types of data related to weather events recorded for a given year: 
1. Storm details
2. Fatalities
3. Locations 

Naming convention for the csv files are as follows: "StormEvents_[file_type]-ftp_v1.0_d[data_year]_c[creation_date].csv.gz"

In [1]:
# Import necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
# Set data paths and columns of interest

LOCATIONS_2021 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_locations-ftp_v1.0_d2021_c20221116.csv.gz"
DETAILS_2021 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2021_c20221116.csv.gz"
FATALITIES_2021 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_fatalities-ftp_v1.0_d2021_c20221116.csv.gz"

LOCATIONS_2020 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_locations-ftp_v1.0_d2020_c20220816.csv.gz"
DETAILS_2020 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2020_c20220816.csv.gz"
FATALITIES_2020 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_fatalities-ftp_v1.0_d2020_c20220816.csv.gz"

LOCATIONS_2019 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_locations-ftp_v1.0_d2019_c20220425.csv.gz"
DETAILS_2019 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2019_c20220425.csv.gz"
FATALITIES_2019 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_fatalities-ftp_v1.0_d2019_c20220425.csv.gz"

LOCATIONS_2018 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_locations-ftp_v1.0_d2018_c20220425.csv.gz"
FATALITIES_2018 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_fatalities-ftp_v1.0_d2018_c20220425.csv.gz"
DETAILS_2018 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2018_c20220425.csv.gz"

LOCATIONS_2017 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_locations-ftp_v1.0_d2017_c20220719.csv.gz"
FATALITIES_2017 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_fatalities-ftp_v1.0_d2017_c20220719.csv.gz"
DETAILS_2017 = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2017_c20220719.csv.gz"

COLS_OF_INTEREST = [
    "BEGIN_YEARMONTH", "END_YEARMONTH", "EVENT_TYPE", "CZ_TYPE", "EPISODE_NARRATIVE", 
    "EVENT_NARRATIVE", "INJURIES_DIRECT", "DEATHS_DIRECT", "DAMAGE_PROPERTY", "DAMAGE_CROPS", 
    'TOR_F_SCALE','TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
    'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME'
]

In [3]:
# Helper functions 

def merge_dfs(df1, df2, col):
    """Merge two dataframes on a common column"""
    joined_df = pd.merge(df1, df2, on=col)
    return joined_df

def year_data_import(loc_csv, detail_csv, fatal_csv, merge_on="EVENT_ID"): 
    """Returns df of merged location, detail, and fatalities csv files 
    for a given year (default is merge on event ID)"""
    loc_df = pd.read_csv(loc_csv)
    detail_df = pd.read_csv(detail_csv)
    fatal_df = pd.read_csv(fatal_csv)
    
    loc_detail_df = merge_dfs(loc_df, detail_df, merge_on)
    joined_df = merge_dfs(loc_detail_df, fatal_df, merge_on)
    
    return joined_df

def missing_data_eval(df, include_col='all'): 
    """Evaluates missing data by row and columns for given dataframe and prints output.
    Specify list of columns to include in include_col, otherwise will include all"""
    if include_col != 'all':
        df = df[include_col]  # specify subset to evaluate 
        
    print("Shape:", df.shape)
    print("\nData missing per col:", df.isnull().sum().sort_values(ascending=False), sep="\n")
    print("\nNumber of rows (right) missing X columns (left)):", df.isnull().sum(axis=1).value_counts(), sep="\n")
    
def change_col_types(df, col, data_type):
    """Change column data types as necessary"""
    df[col] = df[col].astype(data_type)
    return df[col]

def split_date(df, date_col_name, new_names):
    """Splits yearmonth column into separate year and month columns.
    Returns updated df with given naming convention 
    (new_names: list in order of year, month col names)"""
    df[new_names[0]] = df[date_col_name].str[:4]
    df[new_names[1]] = df[date_col_name].str[4:]
    return df

def get_unique_col_vals(df):
    """Gather unique values from each dataframe column"""
    for col in df:
        print(df[col].unique())
        
def identify_data_dims(df):
    """Investigate the dimensions of a dataframe"""
    data_dims = df.shape
    return data_dims
    
def perform_deletion(df):
    """Delete rows with missing variables""" 
    df = df.dropna()
    return df

Consulted Statistics Globe to confirm the process of merging two Pandas DataFrames
https://statisticsglobe.com/merge-pandas-dataframes-based-on-particular-column-python#:~:text=Within%20this%20function%2C%20we%20have%20to%20specify%20the,on%20%3D%20%22col%22%29%20print%28data_join%29%20%23%20Print%20merged%20DataFrame

In [4]:
# Merge data type files for each year and concatenate into on df

df_2021 = year_data_import(LOCATIONS_2021, DETAILS_2021, FATALITIES_2021)
df_2020 = year_data_import(LOCATIONS_2020, DETAILS_2020, FATALITIES_2020)
df_2019 = year_data_import(LOCATIONS_2019, DETAILS_2019, FATALITIES_2019)
df_2018 = year_data_import(LOCATIONS_2018, DETAILS_2018, FATALITIES_2018)
df_2017 = year_data_import(LOCATIONS_2017, DETAILS_2017, FATALITIES_2017)

joined_df = pd.concat([df_2021, df_2020, df_2019, df_2018, df_2017])  # merge into one df

display(joined_df)

,YEARMONTH,EPISODE_ID_x,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,...,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,EVENT_YEARMONTH
0,202102,155272,936331,1,2.96,SSW,KAHAKULOA,20.9611,-156.5692,2057666,...,202102,18,0,42429,I,02/18/2021 00:00:00,26.0,F,In Water,202102.0
1,202102,155272,936331,2,1.58,N,KIPAHULU,20.6725,-156.0739,2040350,...,202102,18,0,42429,I,02/18/2021 00:00:00,26.0,F,In Water,202102.0
2,202102,155279,936389,1,1.60,NW,WAIHEE,20.9455,-156.5385,2056730,...,202102,27,0,42430,I,02/27/2021 00:00:00,27.0,M,In Water,202102.0
3,202102,155279,936389,1,1.60,NW,WAIHEE,20.9455,-156.5385,2056730,...,202102,27,0,42431,I,02/27/2021 00:00:00,NaN,M,In Water,202102.0
4,202102,155279,936389,2,1.01,NNW,KIPAHULU,20.6640,-156.0743,2039840,...,202102,27,0,42430,I,02/27/2021 00:00:00,27.0,M,In Water,202102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,201708,119746,719497,6,35.88,SSW,HARTBURG,29.7479,-93.9029,2944874,...,201708,30,0,33852,D,08/30/2017 00:00:00,61.0,F,Vehicle/Towed Trailer,201708.0
1022,201708,119746,719497,7,13.27,S,HARTBURG,30.0598,-93.7189,303588,...,201708,31,0,33851,D,08/31/2017 00:00:00,68.0,M,In Water,201708.0
1023,201708,119746,719497,7,13.27,S,HARTBURG,30.0598,-93.7189,303588,...,201708,30,0,33852,D,08/30/2017 00:00:00,61.0,F,Vehicle/Towed Trailer,201708.0
1024,201708,119746,719740,1,0.00,N,JASPER,30.9200,-94.0000,3055200,...,201708,31,0,33884,D,08/31/2017 00:00:00,43.0,F,Vehicle/Towed Trailer,201708.0


In [5]:
# Evaluate missing data counts for columns of interest and rows 
missing_data_eval(joined_df, include_col=COLS_OF_INTEREST)


Shape: (3479, 17)

Data missing per col:
TOR_OTHER_CZ_NAME     3145
TOR_OTHER_CZ_FIPS     3145
TOR_OTHER_CZ_STATE    3145
TOR_OTHER_WFO         3145
TOR_F_SCALE           2955
TOR_WIDTH             2955
TOR_LENGTH            2955
DAMAGE_PROPERTY        242
DAMAGE_CROPS           124
EVENT_NARRATIVE          2
END_YEARMONTH            0
DEATHS_DIRECT            0
INJURIES_DIRECT          0
EPISODE_NARRATIVE        0
CZ_TYPE                  0
EVENT_TYPE               0
BEGIN_YEARMONTH          0
dtype: int64

Number of rows (right) missing X columns (left)):
7    2881
1     166
0     156
4     150
9      44
8      30
6      20
5      20
2      12
dtype: int64


Missing data shows that tornado specific feature columns ("TOR_\*")are missing from many of the records, due to not being applicable for event type. Property damage and crop damage variables are only other features of interest missing values.

In [6]:
# Filter the joined dataframe based on the columns of interest
filtered_df = joined_df[COLS_OF_INTEREST]
display(filtered_df.head(5))

print(filtered_df.dtypes)

,BEGIN_YEARMONTH,END_YEARMONTH,EVENT_TYPE,CZ_TYPE,EPISODE_NARRATIVE,EVENT_NARRATIVE,INJURIES_DIRECT,DEATHS_DIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME
0,202102,202102,Heavy Rain,C,"A surface front stalled near Kauai, along with...",A 26-year-old woman died when she was swept ou...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202102,202102,Heavy Rain,C,"A surface front stalled near Kauai, along with...",A 26-year-old woman died when she was swept ou...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202102,202102,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202102,202102,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202102,202102,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN


BEGIN_YEARMONTH         int64
END_YEARMONTH           int64
EVENT_TYPE             object
CZ_TYPE                object
EPISODE_NARRATIVE      object
EVENT_NARRATIVE        object
INJURIES_DIRECT         int64
DEATHS_DIRECT           int64
DAMAGE_PROPERTY        object
DAMAGE_CROPS           object
TOR_F_SCALE            object
TOR_LENGTH            float64
TOR_WIDTH             float64
TOR_OTHER_WFO          object
TOR_OTHER_CZ_STATE     object
TOR_OTHER_CZ_FIPS     float64
TOR_OTHER_CZ_NAME      object
dtype: object


Consulted stack overflow for making a new column from the string slice of another
https://stackoverflow.com/questions/25789445/pandas-make-new-column-from-string-slice-of-another-column

In [7]:
# Change BEGIN_YEARMONTH and END_YEARMONTH from objects to strings to allow for slicing
dtype_dict = {"BEGIN_YEARMONTH":pd.StringDtype(), "END_YEARMONTH": pd.StringDtype()}
filtered_df = filtered_df.astype(dtype_dict)

# Separate BEGIN_YEARMONTH and END_YEARMONTH into year and month columns
filtered_df = split_date(filtered_df, "BEGIN_YEARMONTH", ["BEGIN_YEAR", "BEGIN_MONTH"]) 
filtered_df = split_date(filtered_df, "END_YEARMONTH", ["END_YEAR", "END_MONTH"]) 

# Drop the now-unnecessary columns BEGIN_YEARMONTH and END_YEARMONTH
filtered_df = filtered_df.drop(["BEGIN_YEARMONTH", "END_YEARMONTH"], axis=1)

print(filtered_df.columns)
display(filtered_df.head(5))


Index(['EVENT_TYPE', 'CZ_TYPE', 'EPISODE_NARRATIVE', 'EVENT_NARRATIVE',
       'INJURIES_DIRECT', 'DEATHS_DIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO',
       'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME',
       'BEGIN_YEAR', 'BEGIN_MONTH', 'END_YEAR', 'END_MONTH'],
      dtype='object')


,EVENT_TYPE,CZ_TYPE,EPISODE_NARRATIVE,EVENT_NARRATIVE,INJURIES_DIRECT,DEATHS_DIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,BEGIN_YEAR,BEGIN_MONTH,END_YEAR,END_MONTH
0,Heavy Rain,C,"A surface front stalled near Kauai, along with...",A 26-year-old woman died when she was swept ou...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,02,2021,02
1,Heavy Rain,C,"A surface front stalled near Kauai, along with...",A 26-year-old woman died when she was swept ou...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,02,2021,02
2,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,02,2021,02
3,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,02,2021,02
4,Heavy Rain,C,Gusty trade winds helped keep showers moving a...,Two hikers went missing along the Waikamoi Tra...,0,0,0.00K,0.00K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,02,2021,02


In [8]:
# Change the types of certain columns
dtype_dict = {"EVENT_TYPE":pd.StringDtype(), 
              "CZ_TYPE": pd.StringDtype(), 
              "EPISODE_NARRATIVE":pd.StringDtype(), 
              "EVENT_NARRATIVE":pd.StringDtype(),
              "TOR_F_SCALE":pd.StringDtype(), 
              "TOR_LENGTH":float, 
              "TOR_WIDTH":float,
              "TOR_OTHER_WFO":pd.StringDtype(),
              "TOR_OTHER_CZ_STATE":pd.StringDtype(), 
              "TOR_OTHER_CZ_FIPS":float, 
              "TOR_OTHER_CZ_NAME":pd.StringDtype(),
              "BEGIN_YEAR":int,
              "BEGIN_MONTH":int,
              "END_YEAR":int,
              "END_MONTH":int}

filtered_df = filtered_df.astype(dtype_dict)
print(filtered_df.dtypes)


EVENT_TYPE             string
CZ_TYPE                string
EPISODE_NARRATIVE      string
EVENT_NARRATIVE        string
INJURIES_DIRECT         int64
DEATHS_DIRECT           int64
DAMAGE_PROPERTY        object
DAMAGE_CROPS           object
TOR_F_SCALE            string
TOR_LENGTH            float64
TOR_WIDTH             float64
TOR_OTHER_WFO          string
TOR_OTHER_CZ_STATE     string
TOR_OTHER_CZ_FIPS     float64
TOR_OTHER_CZ_NAME      string
BEGIN_YEAR              int64
BEGIN_MONTH             int64
END_YEAR                int64
END_MONTH               int64
dtype: object


In [9]:
# Clean property damage and crop damage columns to convert to numeric data 
# Replace "K" with (3 zeros - 2), "M" with (6 - 2), 
# "B" with (9 - 2), and "." with "" for DAMAGE_PROPERTY and DAMAGE_CROPS
# 
# Already 2 0's after a decimal point for both columns
letter_num = {"K": "0"*(3-2), "M": "0"*(6-2), "B": "0"*(9-2), ".": ""}
    
filtered_df["DAMAGE_PROPERTY"] \
= filtered_df["DAMAGE_PROPERTY"].str.translate(str.maketrans(letter_num))

filtered_df["DAMAGE_PROPERTY"] = pd.to_numeric(filtered_df["DAMAGE_PROPERTY"])

filtered_df["DAMAGE_CROPS"] \
= filtered_df["DAMAGE_CROPS"].str.translate(str.maketrans(letter_num))

filtered_df["DAMAGE_CROPS"] = pd.to_numeric(filtered_df["DAMAGE_CROPS"])

        
get_unique_col_vals(filtered_df)  # get unique column values for df 

<StringArray>
[              'Heavy Rain',                'Lightning',
              'Flash Flood',                  'Tornado',
        'Thunderstorm Wind',       'Marine Strong Wind',
                    'Flood',              'Debris Flow',
                     'Hail', 'Marine Thunderstorm Wind',
               'Waterspout']
Length: 11, dtype: string
<StringArray>
['C', 'Z']
Length: 2, dtype: string
<StringArray>
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [10]:
# Filter the df by tornadoes, and gauge the missing values in the tornado df
tornado_df = filtered_df[filtered_df["EVENT_TYPE"] == "Tornado"]
display(tornado_df.head(5))

missing_data_eval(tornado_df)


,EVENT_TYPE,CZ_TYPE,EPISODE_NARRATIVE,EVENT_NARRATIVE,INJURIES_DIRECT,DEATHS_DIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,BEGIN_YEAR,BEGIN_MONTH,END_YEAR,END_MONTH
14,Tornado,C,A tornado that touched down in Jefferson Count...,An EF-1 tornado was confirmed to have occurred...,1,1,20000.0,0.0,EF1,0.58,260.0,<NA>,<NA>,NaN,<NA>,2021,10,2021,10
15,Tornado,C,A tornado that touched down in Jefferson Count...,An EF-1 tornado was confirmed to have occurred...,1,1,20000.0,0.0,EF1,0.58,260.0,<NA>,<NA>,NaN,<NA>,2021,10,2021,10
150,Tornado,C,The remnants of Hurricane Ida impacted the mid...,A semi-discrete supercell thunderstorm began t...,2,1,5000000.0,0.0,EF2,8.28,400.0,<NA>,<NA>,NaN,<NA>,2021,9,2021,9
151,Tornado,C,The remnants of Hurricane Ida impacted the mid...,A semi-discrete supercell thunderstorm began t...,2,1,5000000.0,0.0,EF2,8.28,400.0,<NA>,<NA>,NaN,<NA>,2021,9,2021,9
154,Tornado,C,A very moist and moderately unstable airmass o...,A National Weather Service survey determined t...,0,0,20000000.0,NaN,EF4,21.72,1850.0,FFC,GA,113.0,FAYETTE,2021,3,2021,3


Shape: (524, 19)

Data missing per col:
TOR_OTHER_WFO         190
TOR_OTHER_CZ_NAME     190
TOR_OTHER_CZ_FIPS     190
TOR_OTHER_CZ_STATE    190
DAMAGE_PROPERTY       184
DAMAGE_CROPS           66
EVENT_TYPE              0
END_YEAR                0
BEGIN_MONTH             0
BEGIN_YEAR              0
TOR_LENGTH              0
TOR_WIDTH               0
CZ_TYPE                 0
TOR_F_SCALE             0
DEATHS_DIRECT           0
INJURIES_DIRECT         0
EVENT_NARRATIVE         0
EPISODE_NARRATIVE       0
END_MONTH               0
dtype: int64

Number of rows (right) missing X columns (left)):
1    166
0    156
4    150
6     20
5     20
2     12
dtype: int64


In [11]:
# Perform deletion and observe tornado_df's dimensions 
# before and after
initial_dims = identify_data_dims(tornado_df)
print("Below are the initial dimensions of tornado_df.")
print(initial_dims)
print(" ")

tornado_df = perform_deletion(tornado_df)
final_dims = identify_data_dims(tornado_df)
print("Below are the dimensions of tornado_df after "
      "missing values are deleted.")
print(final_dims)
print(" ")

print("After deletion,", initial_dims[0] - final_dims[0], "rows "
      "were removed from the data.")

Below are the initial dimensions of tornado_df.
(524, 19)
 
Below are the dimensions of tornado_df after missing values are deleted.
(156, 19)
 
After deletion, 368 rows were removed from the data.
